# IFT888 - Question 3 partie 3

* Distance de levenshtein


In [8]:
import numpy as np
from typing import List, Tuple

Premièrement, définissons la distance de Levenshtein de façon récursive:

In [9]:
def levenshtein_recursive(a, b):
    if len(b) == 0:
        return len(a)
    if len(a) == 0:
        return len(b)
    if a[0] == b[0]:
        return levenshtein_recursive(a[1:], b[1:])
    return 1 + min(
        levenshtein_recursive(a[1:], b), 
        levenshtein_recursive(a, b[1:]), 
        levenshtein_recursive(a[1:], b[1:]))

In [10]:
a = '01010'
b = '01101'
print(levenshtein_recursive(a, b))

2


Maintenant, définissons la distance de Levenshtein avec l'algorithme X de Wagner-Fischer:

In [11]:
def cost(x, y):
    return 1 if x != y else 0

def levenshtein_memoized(a, b):
    m, n = len(a) + 1, len(b) + 1
    D = np.zeros((m, n), dtype=np.int)
    for i in range(1, m):
        D[i, 0] = D[i-1, 0] + cost(a[i-1], None)

    for j in range(1, n):
        D[0, j] = D[0, j-1] + cost(None, b[j-1])
    for j in range(1, n):
        for i in range(1, m):
            m1 = D[i-1, j] + cost(a[i-1], None)
            m2 = D[i, j-1] + cost(None, b[j-1])
            m3 = D[i-1, j-1] + cost(a[i-1], b[j-1])
            D[i, j] = min(m1, m2, m3)
    return D[m-1, n-1], D

In [12]:
a = '01010'
b = '01101'
# a = 'sitting'
# b = 'kitten'
distance, D = levenshtein_memoized(a, b)
print(D)
print(distance)

[[0 1 2 3 4 5]
 [1 0 1 2 3 4]
 [2 1 0 1 2 3]
 [3 2 1 1 1 2]
 [4 3 2 1 2 1]
 [5 4 3 2 1 2]]
2


L'algorithme X permet d'obtenir la distance, mais l'algorithme Y est requis pour obtenir la "trace":

In [13]:
def levensthein_trace(D: np.ndarray, a: str, b: str) -> Tuple[List, str]:
    i, j = len(a), len(b)
    a_ = list(a)
    ops = []
    while i and j:
        if D[i,j] == D[i-1, j] + cost(a[i-1], None):
            ops += ["Supprésion de {} en position {}".format(a[i-1], i)]
            del a_[i-1]
            i -= 1
        elif D[i,j] == D[i, j-1] + cost(None, b[j-1]):
            ops += ["Ajout de {} en position {} ".format(b[j-1], i)]
            a_.insert(i-1, b[j-1])
            j -= 1
        else:
            if a[i-1] != b[j-1]:
                ops += ["Subsitution de {} par {} en position {}".format(a[i-1], b[j-1], i)]
                a_[i-1] = b[j-1]
            i -= 1
            j -= 1
    return ops[::-1], ''.join(a_)

In [14]:
print('La chaîne de départ est "{}"'.format(a))
ops, a_ = levensthein_trace(D, a, b)
for o in ops:
    print(o)
print('La chaîne finale est "{}"'.format(a_))

La chaîne de départ est "01010"
Ajout de 1 en position 2 
Supprésion de 0 en position 5
La chaîne finale est "01101"
